In [ ]:
# path variables
import sys
project_path = '/Users/naresh/Downloads/DS/growth/nsl_v2/nsl_v2_final/'
sys.path.insert(0, project_path+'config')
from config import SQLQuery

# core libraries
import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
q = SQLQuery('snowflake')

In [ ]:
# north star customers definition
txn_days = 90
txn_credit_amount = 15000

#### Training Dataset

In [ ]:
# query to fetch the required data
df_raw = q("""with 

TRANSACTIONS AS -- get all credit transactions
(
    select BUSINESS_ID,
           TRANSACTION_DATE,
           AMOUNT 
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1,2,3
),

BUSINESSES as -- get all business accounts created between July 2021 and Sep 2022
(
  select *
  from "PROD_DB"."DATA"."BUSINESSES"
  where ACCOUNT_CREATE_DATE between '2021-07-01' and '2022-10-31'
  and business_id not in (select distinct business_id
                          from "PROD_DB"."DATA"."BUSINESSES"
                          where DATEDIFF(day, ACCOUNT_CREATE_DATE, coalesce(ACCOUNT_CLOSE_DATE, current_date)) <= 180
                          and closure_reason in ('Customer Closure Request', 'Customer Request'))
),

ALLOY_PERSONS as -- get all the alloy persons data (max of application_versions_id in case of duplicates)
(
  select * from (
select * ,
       row_number() over (partition by APPLICATION_ID order by APPLICATION_VERSION_ID desc) as rank
from "PROD_DB"."DATA"."ALLOY_EVALUATIONS_PERSONS") 
where rank=1
),

APPLICATIONS as -- get all entries from applications table
(
  select *
  from "PROD_DB"."DATA"."APPLICATIONS"
),

BASE AS -- create base table with business and transactions
(
    select b.BUSINESS_ID,b.APPLICATION_ID,sum(t.amount) as total_credit_amount
    from BUSINESSES b 
    inner join TRANSACTIONS t
    on b.BUSINESS_ID=t.BUSINESS_ID
    where DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) between 0 AND """ + str(txn_days) + """
    group by 1,2
)

    select B.business_id, 
    coalesce(total_credit_amount,0) as total_deposit, 
    A.*, P.*
    from BUSINESSES B
    left join base
    on B.business_id = base.business_id
    left join APPLICATIONS A
    on B.application_id = A.application_id
    left join ALLOY_PERSONS P
    on B.application_id = P.application_id
    """)

df_raw.shape

In [ ]:
df_raw = df_raw.set_index('business_id')

In [ ]:
# Plaid revenue adding to the Novo deposits
df_plaid = pd.read_csv(project_path+'data/plaid_revenue_3m_july21_oct22.csv')
df_plaid.rename(columns={'BUSINESS_ID':'business_id','REVENUE_3M_PLAID':'revenue_3m_plaid'},inplace=True)
df_plaid = df_plaid.set_index('business_id')


In [ ]:
# If in case of change in the data vintage, please use the below link to fetch the plaid data for the mentioned period:
#     https://app.snowflake.com/us-east-1/hba10191/w1Vq4nP3CjUR#query

In [ ]:
df_raw = pd.merge(df_raw, df_plaid[['revenue_3m_plaid']], on='business_id', how='left')
df_raw['revenue_3m_plaid'] = df_raw['revenue_3m_plaid'].fillna(0)
df_raw['total_deposit'] = df_raw['total_deposit'] + df_raw['revenue_3m_plaid']

In [ ]:
# drop null application ids
df_raw = df_raw.dropna(subset='application_id')

# tag north star customers
df_raw['ns_flag'] = np.where(df_raw['total_deposit']>=txn_credit_amount, 1, 0)

# get non-ns vs ns accounts split
df_raw['ns_flag'].value_counts(dropna=False)

In [ ]:
# drop duplicate columns
df_raw = df_raw.loc[:,~df_raw.columns.duplicated()].copy()
df_raw.shape

In [ ]:
print('Applications with $0 deposit:', df_raw[df_raw['total_deposit'] == 0].shape[0])
print('Applications with deposit less than 5K :', df_raw[(df_raw['total_deposit'] <= 5000)].shape[0])
print('Applications with $15K deposit or more:', df_raw[df_raw['total_deposit'] >= txn_credit_amount].shape[0])
print('Applications with deposit between $0 and $15K :', df_raw[(df_raw['total_deposit'] > 0) & (df_raw['total_deposit'] < txn_credit_amount)].shape[0])


In [ ]:
# save the dataset
file = 'nsl_raw_dataset_' + str(datetime.date.today()) + '.pkl'
path = project_path + 'data/'
df_raw.reset_index(inplace=True)
df_raw.to_pickle(path+file)

##### Segment Data

In [ ]:
segment_raw_data = q(
"""
with 
BUSINESS_DETAIL as -- get all business accounts created between July 2021 and Oct 2022
(
  select a.*
  from "PROD_DB"."DATA"."BUSINESSES" a
  inner join
  "PROD_DB"."DATA"."APPLICATIONS" b
  on a.application_id=b.application_id
  where 1=1 
        and date(a.ACCOUNT_CREATE_DATE) between '2021-07-01' and '2022-10-31'
        and business_id not in (
        select distinct business_id
              from "PROD_DB"."DATA"."BUSINESSES"
              where DATEDIFF(day, ACCOUNT_CREATE_DATE, coalesce(ACCOUNT_CLOSE_DATE, current_date)) <= 180
              and closure_reason in ('Customer Closure Request', 'Customer Request'))

)

,segment_all as (
select b.application_id, a.USER_ID, a.anonymous_id, a.CONTEXT_IP, a.OWNER_ID, context_page_path, screen_width, screen_height, timezone, sent_at, received_at
from BUSINESS_DETAIL b
left join SEGMENT_DB.ONBOARDING_PROD.PAGES a
on a.application_id=b.application_id
order by a.application_id, received_at asc
)

-- Pull all the records which crossed the 13th question  
,segment_till_incoming as (select a.application_id, a.context_page_path, a.received_at from 
(select a.application_id, a.context_page_path, a.received_at, rank() over(partition by a.application_id order by a.received_at asc) as rk
from segment_all a
where a.context_page_path='/app/business-questions/incoming'
) a where rk=1 )

-- Pull customer visited pages till the 13th question
,final as (select
a.application_id, a.USER_ID, a.anonymous_id, a.CONTEXT_IP, a.OWNER_ID, a.context_page_path, a.screen_width, a.screen_height, a.timezone, a.sent_at, a.received_at 
from segment_all a
inner join segment_till_incoming b
on a.application_id=b.application_id and a.received_at <= b.received_at
order by a.application_id, a.received_at asc
)

select * from final order by application_id, received_at asc

"""
)

In [ ]:
# save the oot dataset
file = 'segment_raw_dataset_' + str(datetime.date.today()) + '.pkl'
path = project_path + 'data/'
segment_raw_data.reset_index(inplace=True)
segment_raw_data.to_pickle(path+file)

#### OOT Dataset

In [ ]:
# query to fetch the required data
df_oot = q("""with 

TRANSACTIONS AS -- get all credit transactions
(
    select BUSINESS_ID,
           TRANSACTION_DATE,
           AMOUNT 
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1,2,3
),

BUSINESSES as -- get all business accounts created between Nov 2022 and Feb 2023
(
  select *
  from "PROD_DB"."DATA"."BUSINESSES"
  where ACCOUNT_CREATE_DATE between '2022-11-01' and '2023-02-28'
  and business_id not in (select distinct business_id
                          from "PROD_DB"."DATA"."BUSINESSES"
                          where DATEDIFF(day, ACCOUNT_CREATE_DATE, coalesce(ACCOUNT_CLOSE_DATE, current_date)) <= 180
                          and closure_reason in ('Customer Closure Request', 'Customer Request'))
),

ALLOY_PERSONS as -- get all the alloy persons data (max of application_versions_id in case of duplicates)
(
  select * from (
select * ,
       row_number() over (partition by APPLICATION_ID order by APPLICATION_VERSION_ID desc) as rank
from "PROD_DB"."DATA"."ALLOY_EVALUATIONS_PERSONS") 
where rank=1
),

APPLICATIONS as -- get all entries from applications table
(
  select *
  from "PROD_DB"."DATA"."APPLICATIONS"
),

BASE AS -- create base table with business and transactions
(
    select b.BUSINESS_ID,b.APPLICATION_ID,sum(t.amount) as total_credit_amount
    from BUSINESSES b 
    inner join TRANSACTIONS t
    on b.BUSINESS_ID=t.BUSINESS_ID
    where DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) between 0 AND """ + str(txn_days) + """
    group by 1,2
)

    select B.business_id, coalesce(total_credit_amount,0) as total_deposit, A.*, P.*
    from BUSINESSES B
    left join base
    on B.business_id = base.business_id
    left join APPLICATIONS A
    on B.application_id = A.application_id
    left join ALLOY_PERSONS P
    on B.application_id = P.application_id
    """)

df_oot.shape

In [ ]:
df_oot = df_oot.set_index('business_id')

In [ ]:
df_oot[df_oot.total_deposit>=15000].shape

In [ ]:
# Plaid revenue adding to the Novo deposits
df_plaid_oot = pd.read_csv(project_path+'data/plaid_revenue_3m_oot_nov22_feb23.csv')
df_plaid_oot.rename(columns={'BUSINESS_ID':'business_id','REVENUE_3M_PLAID':'revenue_3m_plaid'},inplace=True)
df_plaid_oot = df_plaid_oot.set_index('business_id')
df_oot = pd.merge(df_oot, df_plaid_oot[['revenue_3m_plaid']], on='business_id', how='left')

df_oot['revenue_3m_plaid'] = df_oot['revenue_3m_plaid'].fillna(0)
df_oot['total_deposit'] = df_oot['total_deposit'] + df_oot['revenue_3m_plaid']

In [ ]:
# If in case of change in the data vintage, please use the below link to fetch the plaid data for the mentioned period:
#     https://app.snowflake.com/us-east-1/hba10191/wW00wchf7hv#query

In [ ]:
# drop null application ids
df_oot = df_oot.dropna(subset='application_id')

# tag north star customers
df_oot['ns_flag'] = np.where(df_oot['total_deposit']>=txn_credit_amount, 1, 0)

# get non-ns vs ns accounts split
df_oot['ns_flag'].value_counts(dropna=False)

In [ ]:
# drop duplicate columns
df_oot = df_oot.loc[:,~df_oot.columns.duplicated()].copy()
df_oot.shape

In [ ]:
print('Applications with $0 deposit:', df_oot[df_oot['total_deposit'] == 0].shape[0])
print('Applications with deposit less than 5K :', df_oot[(df_oot['total_deposit'] <= 5000)].shape[0])
print('Applications with $15K deposit or more:', df_oot[df_oot['total_deposit'] >= txn_credit_amount].shape[0])
print('Applications with deposit between $0 and $15K :', df_oot[(df_oot['total_deposit'] > 0) & (df_oot['total_deposit'] < txn_credit_amount)].shape[0])


In [ ]:
# save the oot dataset
file = 'nsl_oot_dataset_' + str(datetime.date.today()) + '.pkl'
path = project_path + 'data/'
df_oot.reset_index(inplace=True)
df_oot.to_pickle(path+file)

##### Segment Data

In [ ]:
segment_pages_oot = q(
"""
with 
BUSINESS_DETAIL as -- get all business accounts created between Nov 2022 and Feb 2023
(
  select a.*
  from "PROD_DB"."DATA"."BUSINESSES" a
  inner join
  "PROD_DB"."DATA"."APPLICATIONS" b
  on a.application_id=b.application_id
  where 1=1 
        and date(a.ACCOUNT_CREATE_DATE) between '2022-11-01' and '2023-02-28'
        and business_id not in (
        select distinct business_id
              from "PROD_DB"."DATA"."BUSINESSES"
              where DATEDIFF(day, ACCOUNT_CREATE_DATE, coalesce(ACCOUNT_CLOSE_DATE, current_date)) <= 180
              and closure_reason in ('Customer Closure Request', 'Customer Request'))

)

,segment_all as (
select b.application_id, a.USER_ID, a.anonymous_id, a.CONTEXT_IP, a.OWNER_ID, context_page_path, screen_width, screen_height, timezone, sent_at, received_at
from BUSINESS_DETAIL b
left join SEGMENT_DB.ONBOARDING_PROD.PAGES a
on a.application_id=b.application_id
order by a.application_id, received_at asc
)

,segment_till_incoming as (select a.application_id, a.context_page_path, a.received_at from 
(select a.application_id, a.context_page_path, a.received_at, rank() over(partition by a.application_id order by a.received_at asc) as rk
from segment_all a
where a.context_page_path='/app/business-questions/incoming'
) a where rk=1 )

,final as (select
a.application_id, a.USER_ID, a.anonymous_id, a.CONTEXT_IP, a.OWNER_ID, a.context_page_path, a.screen_width, a.screen_height, a.timezone, a.sent_at, a.received_at 
from segment_all a
inner join segment_till_incoming b
on a.application_id=b.application_id and a.received_at <= b.received_at
order by a.application_id, a.received_at asc
)

select * from final order by application_id, received_at asc

"""
)

In [ ]:
# save the oot dataset
file = 'segment_oot_dataset_' + str(datetime.date.today()) + '.pkl'
path = project_path + 'data/'
segment_pages_oot.reset_index(inplace=True)
segment_pages_oot.to_pickle(path+file)